<a href="https://colab.research.google.com/github/yibowang15/Capstone/blob/main/Pre_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow.keras.applications

Import VGG16 Model

In [ ]:
from tensorflow.keras.applications import VGG16

# Load VGG16 model, but exclude the classifer on the top layer
# Weight is pre-trained using imagenet dataset
# Classifier on top layer is excluded because customization needed
base_model = VGG16(weights='imagenet', include_top = False, input_shape= (224, 224, 3))

# Check: print the summary of the model
base_model.summary()

Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_25 (InputLayer)          │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 224, 224, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 224, 224, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 112, 112, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 112, 112, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 56, 56, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 28, 28, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 28, 28, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 7, 7, 512)           │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 14,714,688 (56.13 MB)

 Non-trainable params: 0 (0.00 B)

Modify VGG16 Model Input

In [ ]:
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

# Create a new input parameter, which is (224, 224, 1) to adapt to grayscale images
input_tensor = Input(shape=(224, 224, 1))

# Replace the original (224, 224, 3) input and output to (224, 224, 1)
modified_model = Model(inputs=input_tensor, outputs=base_model(input_tensor))


Modify the weight of the first layer

In [ ]:
import numpy as np

# To get the weights of the first convolutional kernel
conv1_weights = base_model.layers[1].get_weights()

# Check: check original kernel shape before adjustment
print("Original kernel shape:", conv1_weights[0].shape)

# To calculate the average of the RGB color channels, axis = 2 represents the third dimension
new_kernel = np.mean(conv1_weights[0], axis=2, keepdims=True)

# Check: check the new kernel shape after being averaged
print("New kernel shape before adjustment:", new_kernel.shape)

# To ensure that after adjusting the weights, the shape is still compatible
new_kernel = np.reshape(new_kernel, (3, 3, 1, 64))  # 重新设定形状以确保与原始形状匹配

# Check: check the new kernel shape after adjustment
print("Adjusted new kernel shape:", new_kernel.shape)

modified_model.summary()


Original kernel shape: (3, 3, 3, 64)
New kernel shape before adjustment: (3, 3, 1, 64)
Adjusted new kernel shape: (3, 3, 1, 64)


Model: "functional_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_23 (InputLayer)          │ (None, 224, 224, 1)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ vgg16 (Functional)                   │ (None, 7, 7, 512)           │      14,714,688 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 14,714,688 (56.13 MB)

 Non-trainable params: 0 (0.00 B)

Reconstruct new classifier： binary classification

In [ ]:
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.models import Model

# Get the pre-trained VGG16 mode, given modified_model has already been adjusted to one color channel
x = Flatten()(modified_model.output)  # Flatten
x = Dense(4096, activation='relu')(x)  # add the dense(fully connected layer)
x = Dropout(0.5)(x)  # add Dropout layer to reduce overfitting
x = Dense(4096, activation='relu')(x)  # add the dense
x = Dropout(0.5)(x)
x = Dense(1, activation='sigmoid')(x)  # output layer

# Create new model
final_model = Model(inputs=modified_model.input, outputs=x)

Compile the final model


In [ ]:
# Use 'binary_crossentropy' as the loss function
final_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Start Trainning

In [ ]:
# Assume train_data 和 validation_data are pre-processed dataset

# final_model.fit(train_data, epochs=10, validation_data=validation_data)